In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from urllib.parse import urlencode
import requests
from requests import Response
import datetime as dt
from datetime import time, date, timedelta
from dateutil.relativedelta import relativedelta
import itertools
from math import ceil
import scipy.stats as stats

In [7]:
result = pd.read_csv('test_data.csv')
result

,3156,AAM20210118,AAM,2021-01-18,11.8,11.6,11.8.1,11.0,15.999999999996358,10.451182
0,3161,AAM20210125,AAM,2021-01-25,11.60,11.50,11.70,10.70,6.7,10.451182
1,3166,AAM20210201,AAM,2021-02-01,11.00,10.95,11.65,10.85,8.8,10.451182
2,3171,AAM20210208,AAM,2021-02-08,10.95,11.35,11.40,10.45,4.1,10.451182
3,3176,AAM20210222,AAM,2021-02-22,11.00,11.20,11.30,10.70,19.8,10.451182
4,3181,AAM20210301,AAM,2021-03-01,11.20,11.05,11.50,10.75,12.0,10.451182
5,3186,AAM20210308,AAM,2021-03-08,10.70,11.30,11.50,10.70,28.0,10.451182
6,3191,AAM20210315,AAM,2021-03-15,11.00,11.10,11.10,10.70,20.3,10.451182
7,3196,AAM20210322,AAM,2021-03-22,11.10,11.00,11.30,10.60,23.0,10.451182
8,3201,AAM20210329,AAM,2021-03-29,11.00,11.10,11.20,10.90,16.1,10.451182
9,3206,AAM20210405,AAM,2021-04-05,11.20,11.00,11.20,10.50,37.9,10.451182


In [66]:
from ta.momentum import RSIIndicator
counter=0
def RSI(window_para, highband, to_csv):
    lowband = 100 - highband
    result = pd.read_csv('raw-weekly-pricevolume.csv')
    codes = result['code'].unique().tolist()
    for code in codes:
        close_weekly = result[result['code']==code]['close_weekly']
        result.loc[code == result['code'].shift(window_para),'rsi'] = RSIIndicator(close=result['close_weekly'],window=window_para).rsi()
    result.loc[result['code'] == result['code'].shift(1),'return'] = result['close_weekly']/result['close_weekly'].shift(1) - 1
    result = result.dropna()
    result.loc[(result['code'] == result['code'].shift(1))&(result['rsi']>highband),'signal'] = 1
    result.loc[(result['code'] == result['code'].shift(1))&(result['rsi']<lowband),'signal'] = 0
    result.loc[result['code'] != result['code'].shift(1),'signal'] = 0
    result = result.ffill()
    result.loc[(result['signal'].shift(1)==1)&(result['signal']!=1),'counter']=1
    result.loc[result['code'] == result['code'].shift(1),'daily_return'] = result['signal'].shift(1)*result['return']
    result.loc[(result['code'] == result['code'].shift(1))&(result['signal']==1)&(result['signal'].shift(1)==0),'daily_return']=result['daily_return']-0.005
    
    dailyprofit = pd.pivot(result, index = 'date', columns = 'code', values = 'daily_return')
    dailyprofit = dailyprofit.replace(0, np.nan)
    dailyprofit.reset_index(inplace = True)
    day_return = dailyprofit.copy()
    day_return['count'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].count(axis = 1) #exclude date column
    day_return.loc[day_return['count']<5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)*(day_return['count']/5)
    day_return.loc[day_return['count']==0,'profit'] = 0
    day_return.loc[day_return['count']>=5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)
    
    profit = (day_return['profit'] +1).cumprod()
    day_return['cumulative'] = profit
    if to_csv:
        day_return.to_csv('checkrsi.csv')
        result.to_csv('resultrsi.csv')
    cul_arr = day_return['cumulative'].to_list()
    dd_arr = []
    maxcul = 0
    for x in day_return.cumulative.dropna().tolist():
        if x >= maxcul:
            maxcul = x
            dd_arr.append(0)
        else:
            dd_arr.append((maxcul-x)/maxcul)
    print(f'MDD {max(dd_arr)}')
    return float(profit[-1:]), window_para, highband, max(dd_arr), result['counter'].sum()

In [81]:
RSI(22, 70, to_csv = True)

MDD 0.1785402925013637


(4.7953504507894635, 22, 70, 0.1785402925013637, 250.0)

In [77]:
optimize_result = []
for window_para in range(6,30, 2):
    for highband in range(50,80,2):
        x=RSI(window_para, highband, to_csv = False)
        optimize_result.append(x)    
optimize_result = pd.DataFrame(optimize_result)

MDD 0.18716436444843176
MDD 0.19636496738719178
MDD 0.19817143476911298
MDD 0.2038637599054901
MDD 0.2055650087409492
MDD 0.21490241359638107
MDD 0.21587105468909937
MDD 0.21660825230696173
MDD 0.21724837709646322
MDD 0.23310310452165103
MDD 0.23566289805514176
MDD 0.2561974426500864
MDD 0.25980540306873806
MDD 0.257456711113283
MDD 0.2477683067152973
MDD 0.19372748642107035
MDD 0.19659621269616934
MDD 0.20356690042409242
MDD 0.2106471042688746
MDD 0.20611537066690458
MDD 0.21936106842257783
MDD 0.22646321756917878
MDD 0.23812220507679296
MDD 0.2419501234894938
MDD 0.24863255796614392
MDD 0.24499502218182392
MDD 0.2543255146162612
MDD 0.25177080075480096
MDD 0.2778715280241311
MDD 0.2710959315038985
MDD 0.19369711105713022
MDD 0.19850253838141318
MDD 0.2026156037686212
MDD 0.21448025274923987
MDD 0.22488054369143268
MDD 0.23071989240305463
MDD 0.24627783371362172
MDD 0.2482432097926653
MDD 0.24622782748472513
MDD 0.24916312962870799
MDD 0.2505874849291046
MDD 0.26054272717433735
MDD 0.

In [78]:
optimize_result.sort_values(by=0, ascending =False ).head(50)
optimize_result

,0,1,2,3,4
0,3.561066,6,50,0.187164,9317.0
1,3.844459,6,52,0.196365,7392.0
2,3.837579,6,54,0.198171,5972.0
3,3.772624,6,56,0.203864,4886.0
4,3.742983,6,58,0.205565,4095.0
...,...,...,...,...,...
175,4.490809,28,70,0.201635,132.0
176,4.426796,28,72,0.211100,89.0
177,5.012038,28,74,0.219868,61.0
178,5.022865,28,76,0.225763,33.0


In [80]:
optimize_result.sort_values(by=0, ascending =False ).head(50)

,0,1,2,3,4
179,6.324636,28,78,0.178309,19.0
164,5.652366,26,78,0.211514,28.0
160,5.078851,26,70,0.183123,169.0
178,5.022865,28,76,0.225763,33.0
177,5.012038,28,74,0.219868,61.0
149,4.922157,24,78,0.219283,38.0
174,4.905384,28,68,0.192213,195.0
134,4.824396,22,78,0.232535,55.0
130,4.795350,22,70,0.178540,250.0
117,4.732083,20,74,0.186475,162.0
